In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re
import time 

In [2]:
#define functions to convert string with dollar sign and comas to numbers
def fix_num(num):
    return float(re.sub('[^0-9\.]', "", num))

def fix_num_int(num):
    return int(re.sub('[^0-9\.]', "", num))

In [17]:
#read Haris county zip codes
zip_code_data=pd.read_csv("harris_county_zip_codes.csv")
zip_code_list=list(zip_code_data['zip_code'].map(str))

In [18]:
zip_code_list[0]

'77002'

In [3]:
#set up chrome browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\sgkuz\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [19]:
#define empty dataframe to appand data to
har_df = pd.DataFrame(columns = ['mls', 'address', 'bld_sft','lot_sft','price', 'zip'])
#search patterns for building and land square footage
bldgsft_pattern = re.compile(r'Building Sqft')
lotsft_pattern = re.compile(r'Lot Sqft')

#define a subroutine to extract data from house structure and append it to a dataframe
def extract_from_house(house):
    global har_df
    asking_price=fix_num(house.find('div', class_="price").text)
    try:
        bldg_sft=fix_num(house.find('div', class_="mp_features").find(text=bldgsft_pattern).__dict__["previous_sibling"].text)
    except AttributeError:
        bldg_sft=0
    try:
        lot_sft=fix_num(house.find('div', class_="mp_features").find(text=lotsft_pattern).__dict__["previous_sibling"].text)
    except AttributeError:
        lot_sft=0
    addr=house.find('a', class_="address").text
    mls_no=fix_num_int(house.find('div', class_="mpi_mls").text)
    property_zip=current_zip
    har_df = har_df.append({'mls' : mls_no, 'address' : addr, 'bld_sft' : bldg_sft, 'lot_sft': lot_sft, 'price': asking_price, 'zip': current_zip }, ignore_index = True)

# cycle through all Harris county zip codes
for current_zip in zip_code_list:
    time.sleep(10)
    print(current_zip)
    #define starting url for current zip code
    url=f'https://www.har.com/zipcode_{current_zip}/realestate/for_sale'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    #Scrape the houses for the current search page
    houses=soup.find_all('div', class_="prop_item")
        
    for house in houses:
        extract_from_house(house)

    #figure out how many pages we will have to click through
    navigation=soup.find_all('div', class_="pagination")
    number_of_pages=len(navigation[0].ul.find_all('li'))

    #click through pages 2 through the last page 
    for i in range(2, number_of_pages+1):
        nextlink=browser.links.find_by_text(str(i))
        time.sleep(5)
        #figure out ambiguity to what is the next search page link
        if len(nextlink)==1:
            nextlink.click()
        else:
            nextlink[1].click()
        
        #extract houses from the current page and append to a data frame
        html = browser.html
        soup = bs(html, 'html.parser')
        houses=soup.find_all('div', class_="prop_item")
        for house in houses:
            extract_from_house(house)
            
#save scraped results for later use            
har_df.to_csv('har_data.csv',index=False)

77002
77003
77004
77005
77006
77007
77008
77009
77010
77011
77012
77013
77014
77015
77016
77017
77018
77019
77020
77021
77022
77023
77024
77025
77026
77027
77028
77029
77030
77031
77032
77033
77034
77035
77036
77037
77038
77039
77040
77041
77042
77043
77044
77045
77046
77047
77048
77049
77050
77051
77053
77054
77055
77056
77057
77058
77059
77060
77061
77062
77063
77064
77065
77066
77067
77068
77069
77070
77071
77072
77073
77074
77075
77076
77077
77078
77079
77080
77081
77082
77083
77084
77085
77086
77087
77088
77089
77090
77091
77092
77093
77094
77095
77096
77098
77099
77336
77338
77339
77345
77346
77357
77365
77373
77375
77377
77379
77388
77389
77396
77401
77429
77433
77447
77449
77450
77477
77484
77489
77493
77494
77502
77503
77504
77505
77506
77520
77521
77523
77530
77532
77536
77546
77547
77562
77571
77581
77586
77587
77598


In [21]:
# end session and close the browser
browser.quit()